In [1]:
if 'spark' in vars():
  spark.stop()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, count, to_timestamp, monotonically_increasing_id, desc, when, sum as _sum, monotonically_increasing_id
from pyspark.sql.functions import dayofmonth, weekofyear, month, year
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.ml.feature import StandardScaler, VectorAssembler

import os 

import numpy as np

import matplotlib.pyplot as plt
from math import isnan

import multiprocessing

In [3]:
# Count available cores
cores = multiprocessing.cpu_count()
# In this case the amount of executors will be equal to the amount of cores
instances = cores

spark = SparkSession.builder \
          .appName("MoneyLaundering") \
          .config("spark.driver.memory", "3g") \
          .config("spark.executor.memory", "4g") \
          .config("spark.executor.instances", cores) \
          .config("spark.executor.cores", cores//instances) \
          .config("spark.sql.shuffle.partitions", cores) \
          .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
          .config("spark.sql.execution.arrow.enabled", "true") \
          .getOrCreate()

spark.sparkContext.setLogLevel("OFF")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/31 19:35:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
dataframe = spark.read.parquet("src/datasets/my_HI-Small_Trans.parquet", header=True)
dataframe = dataframe.withColumn('id', monotonically_increasing_id())

# Feature computing

## Compute features of the whole dataframe

In [ ]:
from FeatureManager import FeatureManager
manager = FeatureManager(dataframe)
manager.compute_features_of_whole_df()

In [ ]:
laundering = manager.dataframe.filter('is_laundering==1')
non_laundering = manager.dataframe.filter('is_laundering==0')

# Compute features of the graph
The next step is to understand the structure of the different patterns in order to identify further features


<img src="src/images/patterns.png" style="width: 600px">


In order to do that, I thought that the best solution was to process the dataset using GraphFrames, a package for Apache Spark which provides DataFrame-based Graph.

Looking at the image below, it would be beneficial to process certain features for each node in the graph to gain valuable insights into the transactions:
1. **Compute the number of in-out edges (fan-in, fan-out)** <br>
    A transaction involves an exchange between two accounts, and it would be valuable to calculate the connection degrees for each account:
    * In-out degrees for the sender account
    * In-out degrees for the receiver account
    <br><br>
2. **Identify intermediary transactions (scatter-gather)** <br>
    By analyzing the flow of transactions, we can identify intermediary transactions. These are transactions that act as intermediaries, facilitating the movement of funds between multiple accounts
    <br><br>
3. **Detect forwarding transactions** <br>
    An account receives a sum of money and then forwards it to another account
    <br><br>
4. **Check for intermediate transactions between two transactions** <br>
    We can check if certain transactions act as intermediaries between two other transactions
    <br><br>

____

<p style="color: green">Analisi delle transazioni ricorrenti: Identifica transazioni ricorrenti o schemi di transazioni che si verificano frequentemente nel dataset.</p>

Analisi di cicli: Identifica cicli nel grafo, poiché potrebbero indicare flussi di denaro chiusi o riciclaggio di fondi.

<p style="color: yellow">Distanza tra nodi: Calcola la distanza tra nodi nel grafo, ad esempio la distanza più breve tra due account, per comprendere quanto sono collegati.</p>

Centralità dei nodi: Calcola la centralità dei nodi nel grafo, come la centralità di grado, la centralità di intermediazione (betweenness centrality) e la centralità di prossimità. Queste metriche possono aiutare a identificare nodi critici o importanti nel sistema di transazioni.

PageRank: calcola i punteggi di PageRank per ciascun account. Il PageRank è una misura dell'importanza di una pagina web nel motore di ricerca Google, ma può essere applicato anche ai grafici delle transazioni per identificare gli account più importanti.

Rilevamento comunità: identifica i gruppi di account che sono più strettamente collegati tra loro rispetto al resto del grafico. Questo può essere utile per rilevare reti di frodi o reti di riciclaggio di denaro.

Flusso di denaro: Segui il flusso di denaro attraverso i collegamenti del grafo, calcolando i flussi di denaro tra diversi nodi e identificando i percorsi più comuni o significativi.

Analisi delle transazioni anomale: Rileva transazioni anomale utilizzando tecniche di rilevamento delle anomalie come il rilevamento degli outlier e l'apprendimento automatico.

Analisi temporale: Esamina l'andamento temporale delle transazioni per rilevare trend o cambiamenti nel tempo.


Misure di centralità: calcola le misure di centralità come la centralità del grado, la centralità della centralità e la centralità dell'autovettore per ogni account. Queste misure possono aiutare a identificare i conti più importanti nel grafico.

Previsione dei collegamenti: utilizza algoritmi di machine learning per prevedere quali account saranno probabilmente connessi in futuro in base alla loro cronologia delle transazioni. Questo può aiutare a identificare potenziali frodi o attività di riciclaggio di denaro prima che si verifichino.

Analisi temporale: analizza i modelli temporali delle transazioni per identificare attività insolite, come un improvviso aumento del numero o della quantità di transazioni.

<p style="color: red">Analisi delle transazioni internazionali: Identifica transazioni che coinvolgono account di diverse nazioni per rilevare attività internazionale.</p>

<p style="color: red">Analisi delle transazioni a valuta: Studia le transazioni che coinvolgono diverse valute per comprendere i flussi di denaro internazionali.</p>


In [5]:
from graphframes import GraphFrame 
from pyspark.sql.functions import lit

class MyGraph:
    # create the graph using the vertices and edges found in the dataset taken into account (train or test)
    def __init__(self, df):
        self.vertices = df.select("from_account")\
                            .withColumnRenamed('from_account', 'id')\
                            .union(df.select("to_account"))\
                            .distinct()

        self.edges = df.withColumnRenamed('from_account', 'src')\
            .withColumnRenamed('to_account', 'dst')

        self.g = GraphFrame(self.vertices, self.edges)
        self.df = df
        self.compute_inOut_degrees()

    def compute_inOut_degrees(self):
        vertexInDegrees = self.g.inDegrees
        vertexOutDegrees = self.g.outDegrees
        vertices = vertexInDegrees.join(vertexOutDegrees, 'id', 'fullouter').fillna(0)

        dataset = self.df.alias('df').join(vertices.alias('vertices'), [self.df.from_account==vertices.id], 'left')\
            .select('df.*', 
                    col('vertices.inDegree').alias('from_account_inDegree'),
                    col('vertices.outDegree').alias('from_account_outDegree'))\
                        .alias('df_from')

        dataset = dataset.join(vertices.alias('vertices'), [self.df.to_account==vertices.id], 'left')\
                .select('df_from.*', 
                    col('vertices.inDegree').alias('to_account_inDegree'),
                    col('vertices.outDegree').alias('to_account_outDegree'))

        self.df = dataset

    def get_forwards(self):
        # it consists in getting all transactions in which the receiver of the transaction 
        # sends the same amount of received money to another account
        # OUTPUT: id of inolved transactions where:
        # - before_forward will be setted to 1 if a transaction is that one before a secondly forwarding transaction
        # - forward will be setted to 1 if a transaction is that one that makes the forward

        motif = "(a)-[e]->(b); (b)-[e2]->(c)"
        forwards = self.g.find(motif).filter("e.amount_received == e2.amount_paid and e.timestamp <= e2.timestamp and a!=b and b!=c")
        forwards = forwards#.select(col('e.id').alias('before_forward'),col('e2.id').alias('forward'))

        before_forward = forwards.select(col('e.id').alias('id'))\
            .distinct()\
            .withColumn('before_forward',lit(1))
        # distinct: I can use it, or I can count how many times the id is involved
        forward = forwards.select(col('e2.id').alias('id'))\
            .distinct()\
            .withColumn('forward',lit(1))
        # distinct: I can use it, or I can count how many times the id is involved
    
        self.df = self.df.join(before_forward, 'id','left').join(forward, 'id','left').na.fill(value=0,subset=['before_forward','forward'])

    def get_distance(self):
        # get the distance between two nodes 
        return 
    
    def same_or_similar(self):
        # it search if for each transaction there is:
        # - another transaction with the same attributes, except the amounts 
        # - another transaction with similar attributes, except the timestamps and amounts
        motif = "(a)-[t1]->(b); (a)-[t2]->(b)"

        same_where = 't1.timestamp == t2.timestamp and \
                        t1.from_bank == t2.from_bank and \
                        t1.to_bank == t2.to_bank and \
                        t1.payment_currency == t2.payment_currency and \
                        t1.receiving_currency == t2.receiving_currency and \
                        t1.payment_format == t2.payment_format and \
                        t1.amount_paid != t2.amount_paid and \
                        t1.id != t2.id'
        
        same = self.g.find(motif).filter(same_where).select('t1.id').withColumn('exist_same',lit(1)).distinct()

        similar_where = 't1.timestamp != t2.timestamp and \
                        t1.from_bank == t2.from_bank and \
                        t1.to_bank == t2.to_bank and \
                        t1.payment_currency == t2.payment_currency and \
                        t1.receiving_currency == t2.receiving_currency and \
                        t1.payment_format == t2.payment_format and \
                        t1.amount_paid != t2.amount_paid'

        
        similar = self.g.find(motif).filter(similar_where).select('t1.id').withColumn('exist_similar',lit(1)).distinct()
        
        self.df = self.df.join(similar, 'id', 'left').join(same, 'id', 'left').na.fill(value=0,subset=['exist_same','exist_similar'])

    def build_temp_ds(self):
        return
    
    def compute_fan_in(self):
        # as explained in undestand_pattern.ipynb it is useful to compute the following feature: 
        # - for each to_account, the number of incoming nodes to the same bank and all in node must have the same: 
        #     * receiving_currency 
        #     * payment_currency
        #     * payment_format
        #     * there must be at most 4 days between the first transaction and the last in the series

        motif = "(a)-[t1]->(b); (c)-[t2]->(b)"
        
        fan_in_query = 'abs(datediff(t1.timestamp, t2.timestamp)) <= 4 and \
                    t1.to_bank == t2.to_bank and \
                    t1.payment_currency == t2.payment_currency and \
                    t1.receiving_currency == t2.receiving_currency and \
                    t1.payment_format == t2.payment_format'
                
        fan_in = self.g.find(motif).filter(fan_in_query).select('a', 'b', 't1')
        fan_in = fan_in.groupBy('a', 'b', 't1').count().select('t1.id',col('count').alias('fan_in_degree'))

        self.df = self.df.join(fan_in, 'id','left')

    def compute_fan_out(self):
        # as explained in undestand_pattern.ipynb it is useful to compute the following feature: 
        # - for each from_account, the number of outgoing nodes to the same bank and all in node must have the same: 
        #     * payment_format
        #     * there must be at most 4 days between the first transaction and the last in the series
        vertices = self.df.select("from_account")\
                        .withColumnRenamed('from_account', 'id')\
                        .union(self.df.select("to_account"))\
                        .distinct()

        edges = self.df.withColumnRenamed('from_account', 'src')\
            .withColumnRenamed('to_account', 'dst').filter('src!=dst and payment_currency==receiving_currency and payment_format=="ACH"')\
            .select('id','timestamp','src','dst','payment_currency','payment_format','from_bank')
        
        g = GraphFrame(vertices, edges)

        motif = "(a)-[t1]->(b); (a)-[t2]->(c)"
        
        fan_out_query = 'abs(datediff(t1.timestamp, t2.timestamp)) <= 4 and \
                        t1.from_bank == t2.from_bank and \
                        a != b and a != c and b != c and\
                        t1.id != t2.id'
                
        fan_out = g.find(motif).filter(fan_out_query).select('a', 'b', 'c', 't1.id')
        fan_out = fan_out.groupBy('a','b','c','id').count()
        fan_out = fan_out.groupBy('id').agg(count('*').alias('fan_out_degree')).select('id', 'fan_out_degree').withColumn('fan_out_degree', col('fan_out_degree')+1)
        
        self.df = self.df.join(fan_out, 'id', 'left').na.fill(value=1,subset=['fan_out_degree'])

    def compute_fan(self):
        self.compute_fan_in()
        self.compute_fan_out()  

In [6]:
my_graph = MyGraph(dataframe)
my_graph.compute_fan()

/Users/simonemalesardi/spark/python/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/Users/simonemalesardi/spark/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [ ]:
my_graph.df.sortBy(col(''))

In [ ]:
def get_connection_degree(df):
    connection_degree = df.groupBy('from_account','to_account').count()

def similar_transaction(df):
    return 

def get_fan(df):
    # count transaction:
    # - made by account in a bank with a currency --> outgoings('from_account','from_bank','payment_currency') -> fan-out
    # - recieved by account in a bank with a currency --> ingoings('to_account','to_bank','receiving_currency') -> fan-in
    outgoings = df.groupBy('from_account','from_bank','payment_currency').count()
    ingoings = df.groupBy('to_account','to_bank','receiving_currency').count()

    df = df.alias('orig').join(outgoings.alias('outgoings'), 
                          [df.from_account==outgoings.from_account,
                           df.from_bank==outgoings.from_bank,
                           df.payment_currency==outgoings.payment_currency], how="left")\
                            .select('orig.*',col('outgoings.count').alias('outgoings'))
    
    df = df.alias('outgoings').join(ingoings.alias('ingoings'), 
                          [df.to_account==ingoings.to_account,
                           df.to_bank==ingoings.to_bank,
                           df.receiving_currency==ingoings.receiving_currency], how="left")\
                            .select('outgoings.*',col('ingoings.count').alias('ingoings'))
    
    return df
    

In [ ]:
my_graph.compute_inOut_degrees()
my_graph.df.show()

In [ ]:
my_graph.same_or_similar()
new_df = get_fan(my_graph.df)

In [ ]:
new_df.show()

In [ ]:
# the example below is an example of "similar" transaction
# dataframe.filter('from_bank==10 and from_account=="800043990" and to_bank==10 and to_account=="800043990" and receiving_currency=="US Dollar"').show()
# the example below is an example of "same" transaction
# dataframe.filter('timestamp=="2022-09-01 00:07:00" and from_bank==1601 and from_account=="8005D0700"').show()

In [ ]:
my_graph.get_forwards()

In [ ]:
my_graph.df.count()

## Fan-in and Fan-out

In [ ]:
dataframe.filter('id==17478').show()
dataframe.filter('from_account=="800737690" and to_account!="800737690" and abs(datediff("2022-09-01 04:33:00",timestamp)) <= 4 and payment_format=="ACH" and to_account!="80020C5B0"').show()

In [ ]:
dataframe.filter('to_account=="812BD4500" and abs(datediff("2022-09-01 00:45:00",timestamp))<=4 and\
                  receiving_currency=="Euro" and payment_format=="Cheque"').show()

In [ ]:
dataframe.filter('to_account=="8001275E0" and abs(datediff("2022-09-05 11:00:00",timestamp))<=4 and\
                  receiving_currency=="US Dollar" and payment_format=="Cheque"').show()